# Homework 0

**Name:** Oscar Eduardo Arámbula Vega

**e-mail:** oscar.arambula4388@alumnos.udg.mx

# MODULES

In [4]:
# Load modules
import numpy as np
import matplotlib.pyplot as plt

# Theory on the Gradient Descent algorithm

In [ ]:
# Function to be optimized


In [ ]:
# Run gradient descent algorithm

In [ ]:
# Plot the results